In [1]:
import csv
import os
import nltk
import random
from nltk import word_tokenize
os.chdir('H:/GEMINI/Results/WATSON/')

In [2]:
f = open('nlp.data.csv', newline = "")
full = csv.reader(f)

In [3]:
full = list(full)
report = []
contrast = []
for row in full:
    report.append(word_tokenize(row[1]))
    contrast.append(row[2])
report = report[1:]
contrast = contrast[1:]

In [4]:
document = [(list(report[i]), contrast[i])
           for i in range(298)]
random.shuffle(document)

In [5]:
len(document)

298

In [6]:
from nltk.corpus import PlaintextCorpusReader
corpus_root = 'H:/GEMINI/Results/WATSON/corpus'
ct_reports = PlaintextCorpusReader(corpus_root, '.*')


In [7]:
all_words = nltk.FreqDist(w.lower() for w in ct_reports.words() if w.isalpha())
word_features = list(all_words)


In [8]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [9]:
featuresets = [(document_features(d), c) for (d,c) in document]
train_set, test_set = featuresets[100:], featuresets[:100]
test = [(a) for (a, b) in featuresets]
classifier = nltk.NaiveBayesClassifier.train(train_set)
classifier.show_most_informative_features(20)

Most Informative Features
          contains(lack) = True                n : y      =     12.9 : 1.0
 contains(opacification) = True                y : n      =      9.0 : 1.0
     contains(deformity) = True                n : y      =      8.4 : 1.0
       contains(quality) = True                y : n      =      6.9 : 1.0
     contains(dimension) = True                n : y      =      6.8 : 1.0
       contains(failure) = True                n : y      =      6.8 : 1.0
    contains(unenhanced) = True                n : y      =      6.8 : 1.0
         contains(gross) = True                n : y      =      6.8 : 1.0
     contains(technique) = True                n : y      =      6.8 : 1.0
       contains(without) = True                n : y      =      6.2 : 1.0
      contains(vascular) = True                n : y      =      5.5 : 1.0
contains(administration) = True                y : n      =      5.4 : 1.0
       contains(annulus) = True                n : y      =      5.3 : 1.0

In [10]:
print(nltk.classify.accuracy(classifier, test_set))

0.74


In [11]:
#classifier.classify_many(test)

In [12]:
classifier2 = nltk.classify.DecisionTreeClassifier.train(train_set, entropy_cutoff = 0.5)

In [13]:
print(nltk.classify.accuracy(classifier2, test_set))

0.85


In [17]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

In [18]:
classif = SklearnClassifier(RandomForestClassifier())
classifier3 = classif.train(train_set)

NameError: name 'SklearnClassifier' is not defined

In [19]:
print(nltk.classify.accuracy(classifier3, test_set))

NameError: name 'classifier3' is not defined

In [20]:
# try different classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

In [21]:
classifiers = [
    SVC(kernel="linear", C=0.02),
    DecisionTreeClassifier(max_depth=6),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=0.1),
    AdaBoostClassifier()
]
for cl in classifiers:
    classif = SklearnClassifier(cl)
    classifier_i = classif.train(train_set)
    print(cl, end = ': ')
    print(nltk.classify.accuracy(classifier_i, test_set))

NameError: name 'SklearnClassifier' is not defined

In [22]:
# parameter tuning for ada
learning_rates = [0.05, 0.1, 0.5, 1, 1.5, 2]
for learning_rate in learning_rates:
    classif = SklearnClassifier(AdaBoostClassifier(n_estimators = 100, learning_rate = learning_rate))
    classifier_i = classif.train(train_set)
    print(learning_rate, end = ': ')
    print(nltk.classify.accuracy(classifier_i, test_set))

NameError: name 'SklearnClassifier' is not defined